# Inter-Rater Agreement
#### Measure the agreement among pairs of raters of the HFC dataset (focusing only on 5/12 raters to simplify the analysis).

In [1]:
from typing import Optional, List

import numpy as np
import pandas as pd
import tqdm

import peyes
import analysis.utils as u
from analysis._article_results.hfc._helpers import *

GT_LABELERS = [GT1, GT2, GT3, GT4, GT5]

In [2]:
dataset = peyes.datasets.hfc(directory=u.DATASETS_DIR, save=False, verbose=True)

In [3]:
def calc_sample_level_agreement(
        dataframe: pd.DataFrame,
        metrics: List[str] = None,
        gt_labelers: List[str] = GT_LABELERS,
        pos_labels: Optional[np.ndarray] = None
) -> pd.DataFrame:
    metrics = metrics or ["balanced_accuracy", "cohen's_kappa", "mcc", "complement_nld"]
    trial_ids = dataframe[peyes.constants.TRIAL_ID_STR].unique()
    results = {}
    for i, trial_id in tqdm.tqdm(enumerate(trial_ids), total=len(trial_ids)):
        trial_data = dataframe[dataframe["trial_id"] == trial_id]
        labels_per_labeler = {lblr: trial_data[lblr].values for lblr in gt_labelers}
        for i, lblr1 in enumerate(labels_per_labeler.keys()):
            for j, lblr2 in enumerate(labels_per_labeler.keys()):
                if i >= j:
                    continue
                res = peyes.sample_metrics.calculate(labels_per_labeler[lblr1], labels_per_labeler[lblr2], *metrics, pos_labels=pos_labels)
                results[(trial_id, lblr1, lblr2)] = res
    results = pd.DataFrame(results).T
    results.index.names = [peyes.constants.TRIAL_ID_STR, u.GT_STR, u.PRED_STR]
    return results

### Entire Dataset
#### (A) All Annotators

In [4]:
overall_agreement = calc_sample_level_agreement(dataset)
overall_agreement_summary = overall_agreement.groupby(level=[u.GT_STR, u.PRED_STR]).describe().stack(0, future_stack=True)
overall_agreement_summary.index.names = [u.GT_STR, u.PRED_STR, "metric"]

overall_agreement_mean = overall_agreement_summary[["mean", "std", "min", "50%", "max"]].groupby(level="metric").mean()
overall_agreement_mean

 17%|█▋        | 12/70 [00:24<01:16,  1.32s/it]C:\Users\jonathanni\PycharmProjects\pEYES\venv\Lib\site-packages\sklearn\metrics\_classification.py:730: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
C:\Users\jonathanni\PycharmProjects\pEYES\venv\Lib\site-packages\sklearn\metrics\_classification.py:730: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
C:\Users\jonathanni\PycharmProjects\pEYES\venv\Lib\site-packages\sklearn\metrics\_classification.py:730: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
100%|██████████| 70/70 [00:47<00:00,  1.47it/s]


,mean,std,min,50%,max
metric,,,,,
balanced_accuracy,0.942564,0.056191,0.613455,0.957421,0.997823
cohen's_kappa,0.830965,0.134615,0.136049,0.854887,0.990621
complement_nld,0.954975,0.035766,0.791490,0.961927,0.999586
mcc,0.837240,0.138835,0.050768,0.860924,0.990658


#### (B) GT1-GT2 Agreement

In [5]:
gt1_gt2_overall_agreement = calc_sample_level_agreement(dataset, gt_labelers=[GT1, GT2])
gt1_gt2_overall_agreement_summary = gt1_gt2_overall_agreement.groupby(level=[u.GT_STR, u.PRED_STR]).describe().stack(0, future_stack=True)
gt1_gt2_overall_agreement_summary.index.names = [u.GT_STR, u.PRED_STR, "metric"]

gt1_gt2_overall_agreement_summary

100%|██████████| 70/70 [00:04<00:00, 14.42it/s]


count      mean       std       min       25%  \
gt pred metric                                                             
IH DN   balanced_accuracy   70.0  0.952527  0.065126  0.500000  0.946946   
        cohen's_kappa       70.0  0.856607  0.133165  0.000000  0.800699   
        mcc                 70.0  0.861905  0.129886  0.000000  0.817095   
        complement_nld      70.0  0.968320  0.025204  0.808415  0.961966   

                                50%       75%  max  
gt pred metric                                      
IH DN   balanced_accuracy  0.965825  0.983680  1.0  
        cohen's_kappa      0.885860  0.934016  1.0  
        mcc                0.887913  0.934514  1.0  
        complement_nld     0.971785  0.980753  1.0

### Free-Viewing Subset
#### (A) All Annotators

In [6]:
fv_dataset = dataset[dataset[peyes.constants.STIMULUS_TYPE_STR] == "free_viewing"]

fv_agreement = calc_sample_level_agreement(fv_dataset)
fv_agreement_summary = fv_agreement.groupby(level=[u.GT_STR, u.PRED_STR]).describe().stack(0, future_stack=True)
fv_agreement_summary.index.names = [u.GT_STR, u.PRED_STR, "metric"]

fv_agreement_mean = fv_agreement_summary[["mean", "std", "min", "50%", "max"]].groupby(level="metric").mean()
fv_agreement_mean

100%|██████████| 10/10 [00:24<00:00,  2.40s/it]


,mean,std,min,50%,max
metric,,,,,
balanced_accuracy,0.922851,0.048410,0.803203,0.937937,0.964052
cohen's_kappa,0.749121,0.112153,0.471965,0.781806,0.846246
complement_nld,0.940056,0.034906,0.850641,0.951405,0.966081
mcc,0.763777,0.104619,0.504275,0.795819,0.852501


#### (B) GT1-GT2 Agreement

In [7]:
gt1_gt2_fv_agreement = calc_sample_level_agreement(fv_dataset, gt_labelers=[GT1, GT2])
gt1_gt2_fv_agreement_summary = gt1_gt2_fv_agreement.groupby(level=[u.GT_STR, u.PRED_STR]).describe().stack(0, future_stack=True)
gt1_gt2_fv_agreement_summary.index.names = [u.GT_STR, u.PRED_STR, "metric"]

gt1_gt2_fv_agreement_summary

100%|██████████| 10/10 [00:02<00:00,  4.05it/s]


count      mean       std       min       25%  \
gt pred metric                                                             
IH DN   balanced_accuracy   10.0  0.944648  0.020104  0.900926  0.939128   
        cohen's_kappa       10.0  0.806712  0.042060  0.764313  0.777827   
        mcc                 10.0  0.813266  0.038667  0.773348  0.786875   
        complement_nld      10.0  0.964497  0.011655  0.935806  0.961901   

                                50%       75%       max  
gt pred metric                                           
IH DN   balanced_accuracy  0.945310  0.959501  0.970974  
        cohen's_kappa      0.781803  0.837265  0.891147  
        mcc                0.793781  0.839614  0.891203  
        complement_nld     0.966570  0.970491  0.978681